<a href="https://colab.research.google.com/github/sethuramcbeboy/CHATBOT-FOR-SUPERMARKET/blob/main/CHATBOT_FOR_SUPERMARKET_SETHURAM_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries


In [ ]:
import csv
import numpy as np
import pandas as pd
import nltk
import string
import random
import sys
import warnings
warnings.filterwarnings("ignore")
nltk.data.path.append("nltk_data")

Import and read the corpus code

In [ ]:
f=open('/content/chat.txt')
raw=f.read()
raw=raw.lower()
nltk.download('punkt')
nltk.download('wordnet')
sent_tokens = nltk.sent_tokenize(raw) # converts to list of sentences
word_tokens = nltk.word_tokenize(raw) # converts to list of words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


example of sentence tokens

In [ ]:
sent_tokens[:2]

['products price quantity\n\ntomato\n\nproduct name=tomato,\nprice/quantity=100/kg,\nstock=100/kg.',
 'potato\n\nproduct name=potato,\nprice/quantity=90/kg,\nstock=120\n********************.']

Example of Word Tokens

In [ ]:
word_tokens[:3]

['products', 'price', 'quantity']

text preprocessing

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


Defining the Greeting Function

In [ ]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up")
GREETING_RESPONSES = ["\nHello, Welcome to the StorePurchasingApp. What do you want?"]

def greeting(sentence):
  for word in sentence.split():
    if word.lower() in GREETING_INPUTS:
      return random.choice(GREETING_RESPONSES)

Response Generations

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def response(user_response):
  robo_response='' 
  TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
  tfidf = TfidfVec.fit_transform(sent_tokens)
  vals = cosine_similarity(tfidf[-1], tfidf)
  idx=vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if(req_tfidf==0):
    robo_response=robo_response+"\nI am sorry! I don't understand you"
    return robo_response
  else:
    robo_response = robo_response+sent_tokens[idx]
    robo_response = robo_response.split("\n",1)[1]
    return robo_response.capitalize()

Importing Table 

Defining conversation start/end protocols

In [ ]:
flag=True
print("BOT: let's start a conversation! not wish just type bye")
while(flag==True):
  user_response = input()
  user_response=user_response.strip()
  user_response=user_response.lower()
  if(user_response!='bye'):
    if(user_response=='thanks' or user_response=='thank you' ):
      flag=False
      print("BOT: You are welcome...")
    else:
      if(greeting(user_response)!=None):
        print("BOT: "+greeting(user_response))
      else:
        sent_tokens.append(user_response)
        word_tokens=word_tokens+nltk.word_tokenize(user_response)
        final_words=list(set(word_tokens))
        print("BOT: ", end = "")
        print(response(user_response))
        sent_tokens.remove(user_response)
  else:
    flag=False
    print("BOT: Bye! Take care ")

BOT: let's start a conversation! not wish just type bye
hi
BOT: 
Hello, Welcome to the StorePurchasingApp. What do you want?
beans
BOT: Product name=cluster beans,
price/quantity=22/kg,
stock=30.
bye
BOT: Bye! Take care 
